In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
import os
os.chdir('D://DKIT//dissertation//Dataset_of_Tomato_Leaves//plantvillage//Preprocessed_data//64//vector//')

In [3]:
#reading numpy array data
pixels = np.load('gray_test.npy')
#reading csv file
data = pd.read_csv('gray_data.csv')

In [4]:
features = pd.DataFrame(pixels)
features = features.values

In [5]:
features

array([[146, 146, 146, ..., 111, 114, 116],
       [174, 173, 171, ..., 132, 129, 128],
       [125, 125, 125, ...,  59,  57,  55],
       ...,
       [120, 120, 121, ..., 130, 129, 127],
       [172, 171, 169, ..., 118, 118, 118],
       [104, 102,  99, ..., 190, 192, 195]], dtype=uint8)

In [6]:
target = data['disease']

In [7]:
cat = target.unique()

In [8]:
vals = list(range(0,10))

In [9]:
for i in range(0,10):
    target.replace(to_replace=cat[i],value = vals[i],inplace = True)

In [10]:
target

0        0
1        0
2        0
3        0
4        0
        ..
14526    9
14527    9
14528    9
14529    9
14530    9
Name: disease, Length: 14531, dtype: int64

In [11]:
target = target.values

In [12]:
target = target.reshape(14531, 1)

In [13]:
target

array([[0],
       [0],
       [0],
       ...,
       [9],
       [9],
       [9]], dtype=int64)

In [14]:
#splitting the data to test and train
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(features,target, test_size=0.20, random_state=24)

In [15]:
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils

In [16]:
xtrain = xtrain.reshape(xtrain.shape[0], 64, 64, 1)
xtest = xtest.reshape(xtest.shape[0], 64, 64, 1)

In [17]:
xtrain = xtrain.astype('float32')
xtest = xtest.astype('float32')

In [18]:
# normalizing the data to help with the training
xtrain /= 255
xtest /= 255

In [19]:
# one-hot encoding using keras' numpy-related utilities
n_classes = 10
ytrain = np_utils.to_categorical(ytrain, n_classes)
ytest = np_utils.to_categorical(ytest, n_classes)

In [20]:
# building a linear stack of layers with the sequential model
gray_cnn_model = Sequential()

# convolutional layer 1
gray_cnn_model.add(Conv2D(50, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=(64, 64, 1)))

# convolutional layer 2
gray_cnn_model.add(Conv2D(75, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
gray_cnn_model.add(MaxPool2D(pool_size=(2,2)))
gray_cnn_model.add(Dropout(0.25))
# convolutional layer 3
gray_cnn_model.add(Conv2D(125, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
gray_cnn_model.add(MaxPool2D(pool_size=(2,2)))
gray_cnn_model.add(Dropout(0.25))

# flatten output of conv
gray_cnn_model.add(Flatten())

# hidden layer
gray_cnn_model.add(Dense(500, activation='relu'))
gray_cnn_model.add(Dropout(0.4))
gray_cnn_model.add(Dense(250, activation='relu'))
gray_cnn_model.add(Dropout(0.3))
# output layer
gray_cnn_model.add(Dense(10, activation='softmax'))

# compiling the sequential model
gray_cnn_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [21]:
gray_cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 50)        500       
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 75)        33825     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 75)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 32, 32, 75)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 125)       84500     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 125)      0         
 2D)                                                    

In [ ]:
# training the model for 10 epochs
test_run = gray_cnn_model.fit(xtrain, ytrain, epochs=5, validation_data=(xtest, ytest))

Epoch 1/5
 67/364 [====>.........................] - ETA: 4:17 - loss: 2.2046 - accuracy: 0.2668

In [ ]:
import matplotlib.pyplot as plt
plt.plot(test_run.history['accuracy'], label='accuracy')
plt.plot(test_run.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
score = gray_cnn_model.evaluate(xtest, ytest, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

In [ ]:
pred = gray_cnn_model.predict(xtest) 
pred = np.argmax(pred, axis = 1)
label = np.argmax(ytest,axis = 1)

In [ ]:
#importing classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(label, pred))
print(confusion_matrix(label, pred))